# Loading Data

Reference - https://python.langchain.com/docs/tutorials/rag/

In [ ]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma
!pip install -U langchain langchain-openai
!pip install sentence-transformers langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 28.7 MB/s eta 0:00:00
  

In [ ]:

# Import packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from transformers import pipeline

In [ ]:
!pip install pyarrow fastparquet huggingface_hub matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.4 MB/s eta 0:00:00


In [ ]:
splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet', 'test': 'yelp_review_full/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.sample(n = 1000, random_state = 43) # using only 1000 rows for PoC to make training quicker

# Retrieval Component

In [ ]:
loader = DataFrameLoader(df, page_content_column = "text")

In [ ]:
# load documents
docs = loader.load()
# split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents = splits, embedding = embeddings)
retriever = vectorstore.as_retriever(search_type = 'similarity')

<ipython-input-8-3c1f5153597d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# HuggingFace pipline
llm_pipeline = pipeline("text-generation", model = "distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
prompt = "Great Italian restaurant with excellent pasta"
retrieved_docs = retriever.get_relevant_documents(prompt)
user_review = prompt

<ipython-input-10-cba8edbc44d0>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(prompt)


In [ ]:
# Display retrieved results
context = ''
print("Retrieved reviews:")
for i, doc in enumerate(retrieved_docs[:5]):
    print(f"Review {i+1}:\n{doc.page_content}\n")
    context += f"Review {i+1}:\n{doc.page_content}\n"

Retrieved reviews:
Review 1:
Sauce by Chef Boyardee. And that is NOT a compliment. The evening started okay. We did not have reservations, but we were early enough and they found a spot for us. The service was impeccable which is why this restaurant has been given 2 stars instead of none. \n\nWe started with champagne. We did not want a full bottle as we were headed to the theater.   The house brand was on the sweet side but okay. Then they brought out the stale bread with the good white beans and olive mixture. The bean stuff was okay but it didn't make up for the lackluster dry bread. I expect so much more from a good Italian. Then we ordered the cheese plate and a sausage plate.   We only raved about one cheese. All of the hard salami choices were good, but it is hard to mess-up that. \n\nThen came the worst main courses ever!  We ordered the pasta because we had heard such great reviews. I had the gnocchi and my friend had the ravioli. The pasta itself was good, but the sauce ruine

In [ ]:
response_prompt = (
    "Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. "
    "The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n"
    f"{retrieved_docs}\n\n"  # Include retrieved reviews for context
    "Please generate a clear and friendly recommendation."
)

In [ ]:
generated_response = llm_pipeline(response_prompt, max_new_tokens=50, num_return_sequences=1)
print("Generated recommendation:")
print('Generated response: ', generated_response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated recommendation:
Generated response:  Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:

[Document(metadata={'label': 1}, page_content="Sauce by Chef Boyardee. And that is NOT a compliment. The evening started okay. We did not have reservations, but we were early enough and they found a spot for us. The service was impeccable which is why this restaurant has been given 2 stars instead of none. \\n\\nWe started with champagne. We did not want a full bottle as we were headed to the theater.   The house brand was on the sweet side but okay. Then they brought out the stale bread with the good white beans and olive mixture. The bean stuff was okay but it didn't make up for the lackluster dry bread. I expect so much more from a good Italian. Then we ordered the cheese plate and a sausage plate.   We only raved about one ch

In [ ]:
print((generated_response)[0])

{'generated_text': 'Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n[Document(metadata={\'label\': 1}, page_content="Sauce by Chef Boyardee. And that is NOT a compliment. The evening started okay. We did not have reservations, but we were early enough and they found a spot for us. The service was impeccable which is why this restaurant has been given 2 stars instead of none. \\\\n\\\\nWe started with champagne. We did not want a full bottle as we were headed to the theater.   The house brand was on the sweet side but okay. Then they brought out the stale bread with the good white beans and olive mixture. The bean stuff was okay but it didn\'t make up for the lackluster dry bread. I expect so much more from a good Italian. Then we ordered the cheese plate and a sausage plate.   We only raved about one cheese. All of the h

# Generation

Look into ChatPrompTemplate to make a custom prompt https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

Models:
Google Gemma [google/gemma-2-2b-it](https://huggingface.co/google/gemma-2-2b-it)

In [ ]:
YELP_PROMPT_TEMPLATE = f"""The following are customer reviews of a business:

BEGIN REVIEWS

{context}

END REVIEWS

Based on the above reviews, provide a personalized recommendation to a user who wrote this review: "{user_review}".

Follow the guidelines below when answering the question:
1. If something is mentioned in multiple reviews, it is important and more likely to be preferred in the answer.
2. If the reviews are of a restaurant, bar, or any food establishment, use specific names of dishes, drinks, and desserts.
"""

print(YELP_PROMPT_TEMPLATE)


def get_template(user_review, context):

  YELP_PROMPT_TEMPLATE = f"""The following are customer reviews of a business:

  BEGIN REVIEWS

  {context}

  END REVIEWS

  Based on the above reviews, provide a personalized recommendation to a user who wrote this review: "{user_review}".

  Follow the guidelines below when answering the question:
  1. If something is mentioned in multiple reviews, it is important and more likely to be preferred in the answer.
  2. If the reviews are of a restaurant, bar, or any food establishment, use specific names of dishes, drinks, and desserts.
  """


  return (YELP_PROMPT_TEMPLATE)


The following are customer reviews of a business:

BEGIN REVIEWS

Review 1:
Sauce by Chef Boyardee. And that is NOT a compliment. The evening started okay. We did not have reservations, but we were early enough and they found a spot for us. The service was impeccable which is why this restaurant has been given 2 stars instead of none. \n\nWe started with champagne. We did not want a full bottle as we were headed to the theater.   The house brand was on the sweet side but okay. Then they brought out the stale bread with the good white beans and olive mixture. The bean stuff was okay but it didn't make up for the lackluster dry bread. I expect so much more from a good Italian. Then we ordered the cheese plate and a sausage plate.   We only raved about one cheese. All of the hard salami choices were good, but it is hard to mess-up that. \n\nThen came the worst main courses ever!  We ordered the pasta because we had heard such great reviews. I had the gnocchi and my friend had the ravioli.

In [ ]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `fafasdasda` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fafasda

In [ ]:
# from langchain import HuggingFaceHub
# from langchain import PromptTemplate, LLMChain


# prompt = PromptTemplate(template=YELP_PROMPT_TEMPLATE, input_variables=["context", "user_review"])

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()
# llm_chain = LLMChain(prompt=prompt,
#                      llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10, "max_length": 100}))

# question = "Great Italian restaurant with excellent pasta"
# response = llm_chain.run(user_review=user_review, context=context)
# print(response)

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",  # replace with "mps" to run on a Mac device
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
messages = [
    {"role": "user", "content": YELP_PROMPT_TEMPLATE},
]

outputs = pipe(messages, max_new_tokens=256)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)


KeyboardInterrupt: 

# **Testing End-to-End Conversational Review Generation**

User review --> Find similar reviews (context) --> Fill in template with review, context --> Pass to model

In [ ]:
def get_prompt(user_review):
  retrieved_docs = retriever.get_relevant_documents(user_review)
  context = ''
  for i, doc in enumerate(retrieved_docs[:5]):
      context += f"Review {i+1}:\n{doc.page_content}\n"
  return get_template(user_review, context)

good_review = 'Great Italian restaurant with excellent pasta'
# bad_review = "I hate Italian food. I hate pizza. Cheese stinks. Pasta is disgusting. The service at the Italian restaurant sucked. I wish I never have to eat Italian food in my life again."
messages = [
    {"role": "user", "content": get_prompt(bad_review)},
]

outputs = pipe(messages, max_new_tokens=256)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)


Based on your strong dislike for Italian food, particularly pizza, cheese, pasta, and the service you experienced, I strongly recommend you steer clear of Italian restaurants altogether.  

It seems like you have a very specific aversion to Italian cuisine.  While I understand that food preferences are subjective, it's important to find places that cater to your tastes. 

Here are some alternative suggestions:

* **Explore other cuisines:**  There are countless other delicious cuisines to discover.  Consider trying Thai, Mexican, Indian, or even American comfort food. 
* **Focus on specific dishes:**  If you're open to trying Italian food again, perhaps you could find a restaurant that specializes in a specific type of Italian cuisine, like Neapolitan pizza or Sicilian pasta. 
* **Look for a different dining experience:**  Maybe you'd enjoy a more casual dining experience, like a food truck or a local pub. 

Remember, food is a personal experience, and it's okay to have preferences.  T